In [1]:
%%capture
!pip install transformers
!pip install bitsandbytes-cuda112
!pip install python-docx

In [2]:
!pip install transformers[deepspeed]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 567 kB 4.0 MB/s 
     |████████████████████████████████| 54 kB 2.8 MB/s 
     |████████████████████████████████| 108 kB 55.1 MB/s 
     |████████████████████████████████| 99 kB 9.9 MB/s 
  Created wheel for deepspeed: filename=deepspeed-0.6.5-py3-none-any.whl size=563643 sha256=6119a80f74e8735aa66f22a7efa7ed41016233c44fcb7f4ad01b8afccbeb1a71
  Stored in directory: /root/.cache/pip/wheels/da/81/aa/058ca61d1510a7144f1ea38897e04c75f5a88f77be3b1c4e7d
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=f3957e9ba3f257d832a90a6bc797ff2f7a1e0d03f809bad0f247267536514971
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
Successfully built deepspeed py-cpuinfo


In [ ]:
!nvidia-smi

Sun Jun 19 18:37:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import torch
from transformers import (
    GPT2Tokenizer, TextDataset, LineByLineTextDataset, 
    DataCollatorForLanguageModeling, TrainingArguments, get_cosine_schedule_with_warmup, BertTokenizer, BertForSequenceClassification, Trainer, GPT2LMHeadModel
)
import os

import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import AutoTokenizer, TrainingArguments, Trainer, GPT2LMHeadModel, GPT2Tokenizer
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import os
import json
import numpy as np
import seaborn as sns
import docx
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Токенизация


In [5]:
special_tokens_dict = {'additional_special_tokens': ['<|query|>', '<|new_row|>',]}
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2", bos_token='<endoftext>', eos_token='<endoftext>', pad_token='<|pad|>')
tokenizer.add_special_tokens(special_tokens_dict)


Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/609 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


2

In [6]:
 bad_words_ids = [[50257]],
        top_p=0.92, 
        temperature=0.9,#0.75
        eos_token_id=50259,
        pad_token_id=50259,
        num_return_sequences=1,
        repition_penalty=10,

IndentationError: ignored

In [ ]:
tokenizer.decode([50257])

'<endoftext>'

In [ ]:
tokenizer.decode([50259])

'<|query|>'

In [7]:
def delete_spaces(s): return ''.join(s.split())
def delete_dublicate_spaces(s): return ' '.join(s.split())

Свои кастомизированные специальные токены можно добавлять и таким способом:

In [8]:
from tqdm import tqdm
import docx
import pandas as pd

class Dataset_processing(Dataset):
    def __init__(self, docs_paths):
        super().__init__()
        self.exmpls = []
        self.max_length = 256
        texts = []
        for doc_path in docs_paths:
            if doc_path.endswith('docx'):
              doc = docx.Document(doc_path)
              text = ''
              for para in doc.paragraphs:
                  if delete_spaces(para.text.lower()) == '***' or '===' in delete_spaces(para.text.lower()):
                      texts.append(delete_dublicate_spaces(text))
                      text = ''
                  elif len(para.text) > 0 and para.text[0] == '>':
                      text += ' <|query|> ' + para.text[1:] + ' <|query|> '
                  elif len(para.text) > 0 and para.text[0] == '-':
                      text +=  ' <|new_row|> ' + para.text
                  else:
                      text += para.text + ' '    
        for text in tqdm(texts,desc='Tokenizing'):
          try:
            encoded = tokenizer.encode(' <endoftext> ' + text + ' <endoftext> ',  truncation=True, max_length=self.max_length, padding="max_length")
            self.exmpls.append(encoded)
          except Exception as e:
            print('error')
            pass
          
    def __len__(self):
        return len(self.exmpls)

    def __getitem__(self, idx):
        return torch.tensor(self.exmpls[idx], dtype=torch.long)

In [9]:
docs_paths = os.listdir('.')

In [10]:
train_dataset = Dataset_processing(docs_paths)

Tokenizing: 100%|██████████| 363/363 [00:03<00:00, 101.49it/s]


# Дообучение модели ruGPT-3


In [11]:
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")

Downloading:   0%|          | 0.00/2.93G [00:00<?, ?B/s]

In [12]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50261, 1536)

In [ ]:
'''
for name, p in model.named_parameters():
    name = name.lower()
    if 'wte' not in name:
        p.requires_grad = False
for name, p in model.lm_head.named_parameters():
    p.requires_grad = True
'''

"\nfor name, p in model.named_parameters():\n    name = name.lower()\n    if 'wte' not in name:\n        p.requires_grad = False\nfor name, p in model.lm_head.named_parameters():\n    p.requires_grad = True\n"

In [13]:
learning_rate = 2e-5
batch_size = 1
num_epochs = 5

In [14]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

seed = 567
training_args = TrainingArguments(
    output_dir='jokes_output',
    overwrite_output_dir=True,
    do_train=True,
    save_steps=10000,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    prediction_loss_only=True,
    logging_steps=100,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    fp16=True,
    #save_steps=300,
    seed=seed,
    learning_rate=learning_rate
)

In [ ]:
import bitsandbytes as bnb
from torch import nn
from transformers.trainer_pt_utils import get_parameter_names

decay_parameters = get_parameter_names(model, [nn.LayerNorm])
decay_parameters = [name for name in decay_parameters if "bias" not in name]

def freeze(
    model,
    freeze_emb=True,
    freeze_ln=False,
    freeze_attn=False,
    freeze_ff=True,
    freeze_other=True,
):
    for name, p in model.named_parameters():
        name = name.lower()
        if 'ln' in name or 'norm' in name:
            p.requires_grad = not freeze_ln
        elif 'embeddings' in name:
            p.requires_grad = not freeze_emb
        elif 'mlp' in name:
            p.requires_grad = not freeze_ff
        elif 'attn' in name:
            p.requires_grad = not freeze_attn
        else:
            p.requires_grad = not freeze_other
    return model

#model = freeze(model)




optimizer_kwargs = {
    "betas": (training_args.adam_beta1, training_args.adam_beta2),
    "eps": training_args.adam_epsilon,
}
optimizer_kwargs["lr"] = training_args.learning_rate
adam_bnb_optim = bnb.optim.Adam8bit(
    model,
    betas=(training_args.adam_beta1, training_args.adam_beta2),
    eps=training_args.adam_epsilon,
    lr=training_args.learning_rate,
)

In [ ]:
model

In [16]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    #optimizers=(adam_bnb_optim, None))
)

train_dataloader = trainer.get_train_dataloader()
num_train_steps = len(train_dataloader) * num_epochs
trainer.create_optimizer_and_scheduler(num_train_steps) #по дефолту в качестве оптимизатора используется AdamW;
                                                        #если вы хотите изменить оптимизатор и/или scheduler, 
                                                        #можно либо передать кортеж, содержащий оптимизатор и  
                                                        #scheduler в init класса Trainer через параметр optimizers,
                                                        #либо создать подкласс и переопределить этот метод в 
                                                        #подклассе
trainer.lr_scheduler = get_cosine_schedule_with_warmup(
    trainer.optimizer,
    num_warmup_steps=num_train_steps//10,
    num_training_steps=num_train_steps
)

Using cuda_amp half precision backend
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [17]:
 trainer.train()

***** Running training *****
  Num examples = 363
  Num Epochs = 5
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 4
  Total optimization steps = 450
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
100,4.210400
200,2.766200
300,2.286500
400,1.962200


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...

TrainOutput(global_step=450, training_loss=2.694227498372396, metrics={'train_runtime': 265.9085, 'train_samples_per_second': 6.826, 'train_steps_per_second': 1.692, 'total_flos': 1892485183832064.0, 'train_loss': 2.694227498372396, 'epoch': 4.99})

# Сохраняем и загружаем дообученную модель

In [ ]:
model.half()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50261, 1536)
    (wpe): Embedding(2048, 1536)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout)

In [ ]:
output_dir = 'potter_large'

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Configuration saved in potter_large/config.json
Model weights saved in potter_large/pytorch_model.bin
tokenizer config file saved in potter_large/tokenizer_config.json
Special tokens file saved in potter_large/special_tokens_map.json
added tokens file saved in potter_large/added_tokens.json


('potter_large/tokenizer_config.json',
 'potter_large/special_tokens_map.json',
 'potter_large/vocab.json',
 'potter_large/merges.txt',
 'potter_large/added_tokens.json')

In [ ]:
[1,2,3,4,5,6][-2:]

[5, 6]

In [ ]:
model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/poets_model7").cuda()

In [18]:
def generate(prompt, length=50):

    model.eval()

    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    input_ids = input_ids.to('cuda')[:, -1 * (256 - length):] #переносим инпут на GPU, где находится наша модель
    #eos_id = tokenizer.encode('<endoftext>')[0]

    sample_outputs = model.generate(
        input_ids, 
        min_length=input_ids.shape[1] + 5,
        max_length=input_ids.shape[1] + length, 
        do_sample=True, 
        top_k=50,
        bad_words_ids = [[50257]],
        top_p=0.92, 
        temperature=0.9,#0.75
        eos_token_id=50259,
        pad_token_id=50259,
        num_return_sequences=1,
        repition_penalty=10,
        num_beams=1
        
        )
    for i, sample_output in enumerate(sample_outputs):
          return tokenizer.decode(sample_output[input_ids.shape[1]:])

In [19]:
def delete_spaces2(s): return ' '.join(s.split())

In [20]:
big_text = ' <endoftext> '
while True:
    text = generate(big_text, length=100)
    #if text.find('<|query2|>') != -1:
    #    text = text[: text.find('<|query2|>') + 1]
    #text = text.replace('<|query2|>', '<|query|>')
    new_texts = text.replace(' <|query|> ', '').split(' <|new_row|> ')
    for new_text in new_texts:
        new_text2 = delete_spaces2(new_text)
        if len(new_text2.replace(' ', '')) > 0:
            print(new_text2)
    print('')
    print('варианты:')
    variants = []
    for i in range(4):
        for j in range(5):
            prompt = big_text + text
            if ' <|query|> ' not in text:
                prompt += ' <|query|> '
            
            variant = delete_spaces2(generate(prompt, length=20).replace(' <|query|> ', ''))
            if ' <|new_row|> ' in variant:
                variant = variant[:variant.find(' <|new_row|> ')]
            if variant not in variants:

                variants.append(variant)
                break
            else:
                continue
        print(variants[i])
        
    print(' ')
    print('напиши, что делать:')
    inp = input() + '<|query|>'
    print(' ')
    if '<|query|>' not in text:
        big_text +=  text + '<|query|>' + inp
    else:
        big_text += text + inp


Гарри и Рон на кухне.
- Знаешь, Рон, я сегодня нахуярился так, что теперь у меня болит всё. <|query|>

варианты:
Предложить, чтобы ты тоже нажрался <|query|>
возмутиться поведением Гарри <|query|>
рияльно ырчать <|query|>
рать на Гарри <|query|>
 
напиши, что делать:
Нажраться
 
роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме роме

варианты:
роме роме роме роме роме роме роме роме роме роме
<|new_row|> - Рон, что у тебя болит? - спрашивает Гарри. <|query|>
<|new_row|> - А ещё я схуярился со своей девушкой, - продолжает Рон. <|query|>
<|new_row|> - Гарри, я вчера нажрался так, что теперь у меня болит всё, -
 
напиши, что делать:


KeyboardInterrupt: ignored

In [ ]:
big_text = '<endoftext>'
while True:
    text = generate(big_text, length=100)
    #if text.find('<|query2|>') != -1:
    #    text = text[: text.find('<|query2|>') + 1]
    #text = text.replace('<|query2|>', '<|query|>')
    new_texts = text.replace('<|query|>', '').split('<|new_row|>')
    for new_text in new_texts:
        new_text2 = delete_spaces2(new_text)
        if len(new_text2.replace(' ', '')) > 0:
            print(new_text2)
    print(' ')
    print('напиши, что делать:')
    inp = input() + '<|query|>'
    print(' ')
    if '<|query|>' not in text:
        big_text += '<|query|>' + text + inp
    else:
        big_text += text + inp


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!zip -r potter_large.zip potter_large/

  adding: potter_large/ (stored 0%)
  adding: potter_large/config.json (deflated 50%)
  adding: potter_large/special_tokens_map.json (deflated 75%)
  adding: potter_large/tokenizer_config.json (deflated 72%)
  adding: potter_large/pytorch_model.bin (deflated 14%)
  adding: potter_large/added_tokens.json (deflated 32%)
  adding: potter_large/vocab.json (deflated 80%)
  adding: potter_large/merges.txt (deflated 76%)
